In [11]:
import os
import sys

from time import perf_counter

# Need to point to where the package is installed
# It will always be in the build directory, but the next folder name might change on your machine
sys.path.append(os.path.abspath('../build/lib.macosx-11.0-arm64-cpython-39'))
import kerrangleconversions as kac

import numpy as np

In [2]:
a = 0.9
p = 10
e = 0.3
x = 0.8
psi = 1.2
chi = 0.5
phi = 10

In [18]:
time = perf_counter()
(Phi_r,Phi_theta,Phi_phi) = kac.pyDarwinPhasesToBoyerLindquistPhases(a,p,e,x,psi,chi,phi)
total_time = (perf_counter() - time) *1000
print("Time taken: %.3f ms" % total_time)
(Phi_r,Phi_theta,Phi_phi)

Time taken: 0.110 ms


(0.7656854348230792, -0.0866817374910737, 9.264252315901594)

In [17]:
time = perf_counter()
(psi2,chi2,phi2) = kac.pyBoyerLindquistPhasesToDarwinPhases(a,p,e,x,Phi_r,Phi_theta,Phi_phi)
total_time = (perf_counter() - time) *1000
print("Time taken: %.3f ms" % total_time)
(psi2,chi2,phi2)

Time taken: 0.308 ms


(1.2000000199292034, 0.5000000255713776, 10.000000009721909)

In [9]:
(psi-psi2,chi-chi2,phi-phi2)

(-1.992920339866089e-08, -2.5571377570976495e-08, -9.721908611481922e-09)